###konlpy, 라이브러리 설치
###데이터 불러오기

In [1]:
# konlpy 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 991 kB/s 
     |████████████████████████████████| 453 kB 72.8 MB/s 


In [2]:
# 라이브러리

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
okt = Okt()
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re


from tqdm.notebook import tqdm

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [11]:
data = pd.read_excel('/content/서울노동위원회 부당해고.xlsx')
data

,Unnamed: 0,사건번호,내용,결과,날짜,관할위원회
0,0,2022부해1317,가. 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,기각,2022-09-07,1
1,1,2022부해1641,가. 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,기각,2022-09-06,1
2,2,2022부해1608,"① 근로자는 구인공고를 보고 입사지원을 하였고, 구인공고에는 “1개월 단기 당직의 ...",기각,2022-09-01,1
3,3,2022부해1595,① 근로자는 관리소장이 ‘지시를 거부하는 근로자는 필요 없다. 다른 사람을 구할 테...,기각,2022-08-31,1
4,4,2022부해1370,가. 근로계약의 법적 성격 ① 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,기각,2022-08-26,1
...,...,...,...,...,...,...
16785,16785,2011부해792,NaN,전부인정,--,1
16786,16786,2012부해425,NaN,각하,--,1
16787,16787,2012부해1556,NaN,전부인정,--,1
16788,16788,2012부해2240,NaN,각하,--,1


In [12]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,사건번호,내용,결과,날짜,관할위원회
0,2022부해1317,가. 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,기각,2022-09-07,1
1,2022부해1641,가. 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,기각,2022-09-06,1
2,2022부해1608,"① 근로자는 구인공고를 보고 입사지원을 하였고, 구인공고에는 “1개월 단기 당직의 ...",기각,2022-09-01,1
3,2022부해1595,① 근로자는 관리소장이 ‘지시를 거부하는 근로자는 필요 없다. 다른 사람을 구할 테...,기각,2022-08-31,1
4,2022부해1370,가. 근로계약의 법적 성격 ① 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,기각,2022-08-26,1
...,...,...,...,...,...
16785,2011부해792,NaN,전부인정,--,1
16786,2012부해425,NaN,각하,--,1
16787,2012부해1556,NaN,전부인정,--,1
16788,2012부해2240,NaN,각하,--,1


In [13]:
data.isnull().sum()

사건번호        0
내용       3472
결과          0
날짜          0
관할위원회       0
dtype: int64

### 데이터 전처리


####제공되지 않은 내용 값(NaN) 날리기

In [42]:
# 16790개 -> 13318개

new_data = data.dropna(inplace=False)
new_data

,사건번호,내용,결과,날짜,관할위원회
0,2022부해1317,가. 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,기각,2022-09-07,1
1,2022부해1641,가. 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,기각,2022-09-06,1
2,2022부해1608,"① 근로자는 구인공고를 보고 입사지원을 하였고, 구인공고에는 “1개월 단기 당직의 ...",기각,2022-09-01,1
3,2022부해1595,① 근로자는 관리소장이 ‘지시를 거부하는 근로자는 필요 없다. 다른 사람을 구할 테...,기각,2022-08-31,1
4,2022부해1370,가. 근로계약의 법적 성격 ① 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,기각,2022-08-26,1
...,...,...,...,...,...
14737,1999부해125,기각,기각,1999-04-08,1
15332,2004부해1076,○ 신청인이 냉장사업부장으로 근무하던 기간 중 부하직원들의 횡령사고에 대한 관리감독...,전부인정,0200-12-09,1
15653,2017부해1339,"... 되는 경우로, ① 홍OO 및 류OO은 비록 4대 보험에 가입되어 있었으나 근...",각하,--,1
15730,2001부해669,신청인의 행위는 징계대상이 된다 할 것이나 폭행당사자가 징계위원으로 참석한것은 공정...,전부인정,--,1


####1차 전처리 (특수문자 등 날리기)

In [43]:
new_data = new_data.copy()
new_data['내용'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s)for s in new_data['내용']]
new_data.reset_index(inplace=True)
new_data

,index,사건번호,내용,결과,날짜,관할위원회
0,0,2022부해1317,가 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,기각,2022-09-07,1
1,1,2022부해1641,가 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,기각,2022-09-06,1
2,2,2022부해1608,근로자는 구인공고를 보고 입사지원을 하였고 구인공고에는 1개월 단기 당직의 ...,기각,2022-09-01,1
3,3,2022부해1595,근로자는 관리소장이 지시를 거부하는 근로자는 필요 없다 다른 사람을 구할 테...,기각,2022-08-31,1
4,4,2022부해1370,가 근로계약의 법적 성격 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,기각,2022-08-26,1
...,...,...,...,...,...,...
13313,14737,1999부해125,기각,기각,1999-04-08,1
13314,15332,2004부해1076,신청인이 냉장사업부장으로 근무하던 기간 중 부하직원들의 횡령사고에 대한 관리감독...,전부인정,0200-12-09,1
13315,15653,2017부해1339,되는 경우로 홍OO 및 류OO은 비록 4대 보험에 가입되어 있었으나 근...,각하,--,1
13316,15730,2001부해669,신청인의 행위는 징계대상이 된다 할 것이나 폭행당사자가 징계위원으로 참석한것은 공정...,전부인정,--,1


####판결 라벨링

  (기각, 전부인정, 각하만)


In [44]:
new_data.loc[new_data['결과'].str.contains('기각'), '라벨링'] = 0
new_data.loc[new_data['결과'].str.contains('전부인정'), '라벨링'] = 1
new_data.loc[new_data['결과'].str.contains('각하'), '라벨링'] = 2

new_data.dropna(inplace=True)

In [45]:
# 라벨링 정수화
new_data['라벨링'] = new_data['라벨링'].astype(int)

In [46]:
new_data.reset_index(drop=True,inplace=True)

In [47]:
new_data

,index,사건번호,내용,결과,날짜,관할위원회,라벨링
0,0,2022부해1317,가 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,기각,2022-09-07,1,0
1,1,2022부해1641,가 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,기각,2022-09-06,1,0
2,2,2022부해1608,근로자는 구인공고를 보고 입사지원을 하였고 구인공고에는 1개월 단기 당직의 ...,기각,2022-09-01,1,0
3,3,2022부해1595,근로자는 관리소장이 지시를 거부하는 근로자는 필요 없다 다른 사람을 구할 테...,기각,2022-08-31,1,0
4,4,2022부해1370,가 근로계약의 법적 성격 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,기각,2022-08-26,1,0
...,...,...,...,...,...,...,...
12813,14737,1999부해125,기각,기각,1999-04-08,1,0
12814,15332,2004부해1076,신청인이 냉장사업부장으로 근무하던 기간 중 부하직원들의 횡령사고에 대한 관리감독...,전부인정,0200-12-09,1,1
12815,15653,2017부해1339,되는 경우로 홍OO 및 류OO은 비록 4대 보험에 가입되어 있었으나 근...,각하,--,1,2
12816,15730,2001부해669,신청인의 행위는 징계대상이 된다 할 것이나 폭행당사자가 징계위원으로 참석한것은 공정...,전부인정,--,1,1


####50글자 미만 내용 없애기(필요한 정보만 뽑아내기)

In [48]:
fin_data = {
    '내용':[],
    '라벨':[]
}

In [49]:
for i in tqdm(range(len(new_data))) :
  if len(list(new_data['내용'].values)[i]) >= 50 :
    fin_data['내용'].append(new_data['내용'][i])
    fin_data['라벨'].append(new_data['라벨링'][i])

fin_data = pd.DataFrame(fin_data)
fin_data

  0%|          | 0/12818 [00:00<?, ?it/s]

,내용,라벨
0,가 사용자 적격 인정 여부 연맹과 노동조합의 대표가 동일하고 주소지가 같으며 통합...,0
1,가 징계해고 사유의 존재 여부 근로자는 공무직으로 서울시 공무직 채용 및 복무 등...,0
2,근로자는 구인공고를 보고 입사지원을 하였고 구인공고에는 1개월 단기 당직의 ...,0
3,근로자는 관리소장이 지시를 거부하는 근로자는 필요 없다 다른 사람을 구할 테...,0
4,가 근로계약의 법적 성격 기간의 정함이 없는 최초 근로계약서 체결 직후 사용자...,0
...,...,...
12631,근로자를 채용하면서 학력 또는 경력을 기재한 이력서의 제출을 요구하는 것은 ...,0
12632,신청인이 냉장사업부장으로 근무하던 기간 중 부하직원들의 횡령사고에 대한 관리감독...,1
12633,되는 경우로 홍OO 및 류OO은 비록 4대 보험에 가입되어 있었으나 근...,2
12634,신청인의 행위는 징계대상이 된다 할 것이나 폭행당사자가 징계위원으로 참석한것은 공정...,1


####2차 전처리 (포스)
- 명사+동사만 추출

In [50]:
def tokenizer(raw, pos=["Noun", "Verb" ] , stopword = []):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]

In [51]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=2,
    sublinear_tf = True
)

In [53]:
pos_data = {
    '내용':[],
     '라벨':[]
}
for i in tqdm(range(len(fin_data))):
  pos_data['내용'].append(tokenizer(fin_data['내용'][i]))
  pos_data['라벨'].append(fin_data['라벨'][i])


  0%|          | 0/12636 [00:00<?, ?it/s]

In [54]:
df_pos_data=pd.DataFrame(pos_data)
df_pos_data

,내용,라벨
0,"[가다, 사용자, 적격, 인정, 여부, 연맹, 노동조합, 대표, 주소지, 통합, 예...",0
1,"[가다, 징계, 해고, 사유, 존재, 여부, 근로자, 무직, 서울시, 무직, 채용,...",0
2,"[근로자, 구인, 공고, 보고, 입사, 지원, 하다, 구인, 공고, 개월, 단기, ...",0
3,"[근로자, 관리소, 지시, 거부, 하다, 근로자, 필요, 다른, 사람, 하다, 만두...",0
4,"[가다, 근로, 계약, 법적, 성격, 기간, 정함, 최초, 근로계약서, 체결, 직후...",0
...,...,...
12631,"[근로자, 채용, 하다, 학력, 경력, 기재, 이력서, 제출, 요구, 하다, 근로자...",0
12632,"[신청, 냉장, 사업, 부장, 근무, 하다, 기간, 부하, 직원, 횡령, 사고, 대...",1
12633,"[되다, 경우, 비록, 대다, 보험, 가입, 되어다, 근로, 제공, 하다, 않다, ...",2
12634,"[신청, 행위, 징계, 대상, 되다, 하다, 폭행, 당사자, 징계, 위원, 참석, ...",1


In [55]:
df_pos_data.to_excel('pos.xlsx', index=False) # 임시 저장

####3차 전처리 (각 행 [], 없애기 - 리스트 형태 해제)
  - 한 글자만 나오는 제외를 위함

In [56]:
df_pos_data['내용'] = df_pos_data['내용'].apply(lambda x: ' '.join(x))
df_pos_data

,내용,라벨
0,가다 사용자 적격 인정 여부 연맹 노동조합 대표 주소지 통합 예정 되어다 하나 사업...,0
1,가다 징계 해고 사유 존재 여부 근로자 무직 서울시 무직 채용 복무 조례 적용 받다...,0
2,근로자 구인 공고 보고 입사 지원 하다 구인 공고 개월 단기 당직 구합 제목 구인 ...,0
3,근로자 관리소 지시 거부 하다 근로자 필요 다른 사람 하다 만두라 하다 해고 하다 ...,0
4,가다 근로 계약 법적 성격 기간 정함 최초 근로계약서 체결 직후 사용자 근로자 행실...,0
...,...,...
12631,근로자 채용 하다 학력 경력 기재 이력서 제출 요구 하다 근로자 능력 평가 하다 하...,0
12632,신청 냉장 사업 부장 근무 하다 기간 부하 직원 횡령 사고 대한 관리 감독 해태 부...,1
12633,되다 경우 비록 대다 보험 가입 되어다 근로 제공 하다 않다 대해 당사자 다툼 늘다...,2
12634,신청 행위 징계 대상 되다 하다 폭행 당사자 징계 위원 참석 정성 결여 되다 정면 ...,1


####4차 전처리 (한 글지 날리기)

In [57]:
# 한 글자만 남기는 함수 정의

def remove(x) :
    m = re.sub(r'(?:\b[0-9a-zA-Zㄱ-ㅎㅏ-ㅣ]\b|[?!\W]+)\s*', ' ', x).strip() 
    m = " ".join(m.split())
    return m

In [58]:
f_data = {'내용' : [],
            '라벨' : []}

for i in tqdm(range(len(df_pos_data))) :
  f_data['내용'].append(remove(str(df_pos_data['내용'][i])))
  f_data['라벨'].append(df_pos_data['라벨'][i])

f_data = pd.DataFrame(f_data)
f_data

  0%|          | 0/12636 [00:00<?, ?it/s]

,내용,라벨
0,가다 사용자 적격 인정 여부 연맹 노동조합 대표 주소지 통합 예정 되어다 하나 사업...,0
1,가다 징계 해고 사유 존재 여부 근로자 무직 서울시 무직 채용 복무 조례 적용 받다...,0
2,근로자 구인 공고 보고 입사 지원 하다 구인 공고 개월 단기 당직 구합 제목 구인 ...,0
3,근로자 관리소 지시 거부 하다 근로자 필요 다른 사람 하다 만두라 하다 해고 하다 ...,0
4,가다 근로 계약 법적 성격 기간 정함 최초 근로계약서 체결 직후 사용자 근로자 행실...,0
...,...,...
12631,근로자 채용 하다 학력 경력 기재 이력서 제출 요구 하다 근로자 능력 평가 하다 하...,0
12632,신청 냉장 사업 부장 근무 하다 기간 부하 직원 횡령 사고 대한 관리 감독 해태 부...,1
12633,되다 경우 비록 대다 보험 가입 되어다 근로 제공 하다 않다 대해 당사자 다툼 늘다...,2
12634,신청 행위 징계 대상 되다 하다 폭행 당사자 징계 위원 참석 정성 결여 되다 정면 ...,1


####stopword 계산을 위한 단어 빈도수 사전

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer() 
dtm = cvect.fit_transform(f_data['내용'])
dtm

<12636x12188 sparse matrix of type '<class 'numpy.int64'>'
	with 694547 stored elements in Compressed Sparse Row format>

In [60]:
feature_names = cvect.get_feature_names_out()
feature_names

array(['가가', '가감', '가게', ..., '힐튼', '힘내다', '힘쓰다'], dtype=object)

In [61]:
vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None, 
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 1,  
                             ngram_range=(1, 1), 
                             max_features = 300000
                            )
vectorizer

CountVectorizer(max_features=300000)

In [62]:
feature_vector = vectorizer.fit_transform(f_data['내용'])
feature_vector.shape

(12636, 12188)

In [63]:
vocab = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [64]:
dist = np.sum(feature_vector, axis=0)
    
df_freq = pd.DataFrame(dist, columns=vocab) 
df_freq_T = df_freq.T.sort_values(by=0, ascending=False)

In [65]:
df_freq_T.reset_index(inplace=True)
df_freq_T.columns = ['단어','횟수']
df_freq_T

,단어,횟수
0,하다,96301
1,근로자,46769
2,사건,31545
3,되다,26364
4,사용자,24772
...,...,...
12183,십만,1
12184,심혈,1
12185,심해지다,1
12186,심정지,1


In [66]:
# 엑셀로 저장

f_data.to_excel('f_data.xlsx', index=False)
df_freq_T.to_excel('df_freq_T.xlsx', index=False)

### stopword 돌리기

In [67]:
data_bin = pd.read_excel('/content/df_freq_T.xlsx')
data_bin

,단어,횟수
0,하다,96301
1,근로자,46769
2,사건,31545
3,되다,26364
4,사용자,24772
...,...,...
12183,십만,1
12184,심혈,1
12185,심해지다,1
12186,심정지,1


In [68]:
# 3번 이하 등장하는 단어들 삭제

data_count = data_bin.loc[data_bin['횟수']<=3]
data_count_bin = list(data_count['단어'].values)
data_count_bin

['주급',
 '근면',
 '갑질',
 '극장',
 '유흥',
 '엄히',
 '강동구청',
 '직판',
 '육안',
 '얹다',
 '엄중',
 '인부',
 '절개',
 '직통',
 '혼재',
 '전환사채',
 '극구',
 '강세',
 '전하',
 '이주하',
 '기화',
 '원함',
 '까르푸',
 '진과',
 '어윤나',
 '정보통신망',
 '금감원',
 '정보통',
 '기지시',
 '양재동',
 '갈아입다',
 '학회',
 '종친회',
 '담합',
 '감경철',
 '감금',
 '왕성',
 '진로',
 '나빠지다',
 '증후군',
 '근저당권',
 '휘두르다',
 '훔치다',
 '왕래',
 '유턴',
 '닫히다',
 '감지',
 '직장폐쇄',
 '월계',
 '단합',
 '유통업',
 '정거',
 '감지기',
 '광고대행사',
 '닥터',
 '엔진오일',
 '홍원',
 '원리금',
 '개축',
 '전도',
 '주사약',
 '전담텔러',
 '은퇴',
 '와이드',
 '이장우',
 '전기요금',
 '객지',
 '다리미',
 '지리',
 '다리',
 '김강',
 '협박죄',
 '다량',
 '회화',
 '적체',
 '거부처분',
 '인스타그램',
 '거소',
 '워크아웃',
 '적응도',
 '증여',
 '권내',
 '허가증',
 '개인정보보호법',
 '전문교육',
 '완납',
 '개월전',
 '전주교대',
 '한식',
 '양육',
 '증축',
 '귀하',
 '원스',
 '업소',
 '윤번제',
 '개년',
 '원성',
 '전시회',
 '전시장',
 '귀중',
 '개봉',
 '다음해',
 '전술',
 '인선',
 '한인',
 '윤전',
 '전산실',
 '에스뻬제',
 '개소',
 '개수',
 '직분',
 '교정',
 '윤태호',
 '다시다',
 '혼합',
 '간혹',
 '인도네시아',
 '가목',
 '가로수',
 '제철소',
 '체납',
 '대수',
 '대성',
 '교육과학기술부',
 '가리키다',
 '가망',
 '가면',
 '지구대',
 '제주도',
 '제조업',
 

In [69]:
# 전처리 데이터 입력
data = pd.read_excel('/content/f_data.xlsx')
data

,내용,라벨
0,가다 사용자 적격 인정 여부 연맹 노동조합 대표 주소지 통합 예정 되어다 하나 사업...,0
1,가다 징계 해고 사유 존재 여부 근로자 무직 서울시 무직 채용 복무 조례 적용 받다...,0
2,근로자 구인 공고 보고 입사 지원 하다 구인 공고 개월 단기 당직 구합 제목 구인 ...,0
3,근로자 관리소 지시 거부 하다 근로자 필요 다른 사람 하다 만두라 하다 해고 하다 ...,0
4,가다 근로 계약 법적 성격 기간 정함 최초 근로계약서 체결 직후 사용자 근로자 행실...,0
...,...,...
12631,근로자 채용 하다 학력 경력 기재 이력서 제출 요구 하다 근로자 능력 평가 하다 하...,0
12632,신청 냉장 사업 부장 근무 하다 기간 부하 직원 횡령 사고 대한 관리 감독 해태 부...,1
12633,되다 경우 비록 대다 보험 가입 되어다 근로 제공 하다 않다 대해 당사자 다툼 늘다...,2
12634,신청 행위 징계 대상 되다 하다 폭행 당사자 징계 위원 참석 정성 결여 되다 정면 ...,1


In [70]:
# 임의로 리스트화한 stopword에 3번 이하로 나온 단어들 추가

stopword = ['으로','에게','하고','여부','되어다','에서','대한','이다','대하','대해','라고','관련','에도','이라고','개월','부터','로서','에는','이고','또한','까지','이나','따라서','장하나','또는','로부터','거나','것이므','으로서','라는',2012,'자로','와의',2013,'통해',2011,'하나',2010,2009,'만으로','정이',2019,2018,'장이','이며','으로써',2008,'것임','하므다','가지','이라',2007,2021,2014,'으로부터','과의',2017,2015,2016,2020,'비다',00,'그러나','항제',24,'이로','각각','에서도','반면','에게는',000,'한편','역시','호에','에서의','까지의','000원','까지로','와는','먼저','해보다','라도','으로서의','일련','에게만','oo','어느','로기','로써','으로의','그리고','것일','부로','첫째','결코','2008년','2009년','그로','2010년','2007년','ooo','2005','20일','8월','10월','11월','이지만','만으로도','000만원','이라고는','0000','나름','00000','그냥','부터의','이라고도','이므로','경에','까지에','이냐','다소간','게다가','00조','이사나','사회자']
 
for i in data_count_bin:
  stopword.append(i)

In [71]:
# 명사와 동사만 남기기

def tokenizer1(raw, pos=["Noun","Verb"] , stopword = stopword):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]


In [72]:
N_V_data = {'내용':[],
            '라벨':[]}

for i in tqdm(range(len(data))) :
  N_V_data['내용'].append(tokenizer1(data['내용'][i]))
  N_V_data['라벨'].append(data['라벨'][i])

  0%|          | 0/12636 [00:00<?, ?it/s]

In [73]:
# 명사 동사 데이터 저장

N_V_data = pd.DataFrame(N_V_data)
N_V_data['내용'] = N_V_data['내용'].apply(lambda x: ' '.join(x))
N_V_data.to_excel('Noun_Verb_data.xlsx', index=False)
N_V_data

,내용,라벨
0,가다 사용자 적격 인정 연맹 노동조합 대표 주소지 통합 예정 사업 연맹 노동조합 독...,0
1,가다 징계 해고 사유 존재 근로자 무직 서울시 무직 채용 복무 조례 적용 받다 징역...,0
2,근로자 구인 공고 보고 입사 지원 하다 구인 공고 단기 당직 제목 구인 공고 하다 ...,0
3,근로자 관리소 지시 거부 하다 근로자 필요 다른 사람 하다 만두라 하다 해고 하다 ...,0
4,가다 근로 계약 법적 성격 기간 정함 최초 근로계약서 체결 직후 사용자 근로자 행실...,0
...,...,...
12631,근로자 채용 하다 학력 경력 기재 이력서 제출 요구 하다 근로자 능력 평가 하다 하...,0
12632,신청 사업 부장 근무 하다 기간 부하 직원 횡령 사고 관리 감독 해태 부하 직원 금...,1
12633,되다 경우 비록 대다 보험 가입 근로 제공 하다 않다 당사자 다툼 늘다 당시 다른 ...,2
12634,신청 행위 징계 대상 되다 하다 폭행 당사자 징계 위원 참석 정성 결여 되다 정면 ...,1


In [74]:
# 명사만 남기기
# 우리가 분석 타겟으로 정한 것

def tokenizer2(raw, pos=["Noun"] , stopword = stopword):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]


In [75]:
V_data = {'내용' : [],
          '라벨' : []}

for i in tqdm(range(len(data))) :
  V_data['내용'].append(tokenizer2(data['내용'][i]))
  V_data['라벨'].append(data['라벨'][i])



  0%|          | 0/12636 [00:00<?, ?it/s]

In [76]:
# 명사 데이터 저장

V_data = pd.DataFrame(V_data)
V_data['내용'] = V_data['내용'].apply(lambda x: ' '.join(x))
V_data.to_excel('Noun_data.xlsx',index=False)
V_data

,내용,라벨
0,사용자 적격 인정 연맹 노동조합 대표 주소지 통합 예정 사업 연맹 노동조합 독립성 ...,0
1,징계 해고 사유 존재 근로자 무직 서울시 무직 채용 복무 조례 적용 징역 집행유예 ...,0
2,근로자 구인 공고 보고 입사 지원 구인 공고 단기 당직 제목 구인 공고 사용자 당직...,0
3,근로자 관리소 지시 거부 근로자 필요 다른 사람 만두라 해고 근로자 제출 증거 당사...,0
4,근로 계약 법적 성격 기간 정함 최초 근로계약서 체결 직후 사용자 근로자 행실 투서...,0
...,...,...
12631,근로자 채용 학력 경력 기재 이력서 제출 요구 근로자 능력 평가 노사 신뢰 형성 기...,0
12632,신청 사업 부장 근무 기간 부하 직원 횡령 사고 관리 감독 해태 부하 직원 금품 수...,1
12633,경우 비록 보험 가입 근로 제공 당사자 다툼 당시 다른 회사 소속 임원 이면 사용자...,2
12634,신청 행위 징계 대상 폭행 당사자 징계 위원 참석 정성 결여 정면 해고 판단,1
